In [37]:
%load_ext lab_black

In [40]:
import kfp

In [41]:
BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:elyra3.11.1-py3.8-tf2.8.1-v0"

In [53]:
def visualize_plot(mlpipeline_ui_metadata_path: kfp.components.OutputPath(str)):
    import base64
    import json

    import matplotlib.pyplot as plt
    import numpy as np
    import random

    # Create a scatter plot around a regression line
    x = np.arange(100)
    delta = np.random.uniform(-5, 5, size=(len(x),))
    m = random.randint(1, 10) / 10
    y = random.randint(0, 5)
    y = m * x + y + delta

    # slove regression
    m, b = np.polyfit(x, y, 1)

    # draw plot and save
    plt.plot(x, y, "o")
    plt.plot(x, m * x + b)
    plt.savefig("regression.png")

    # create the visualization
    # https://www.kubeflow.org/docs/components/pipelines/v1/sdk/output-viewer/#v1-sdk-writing-out-metadata-for-the-output-viewers
    with open("regression.png", "rb") as f:
        jpg = base64.b64encode(f.read())

    html = f'<img src="data:image/jpg;base64,{jpg.decode("utf-8")}"/>'
    metadata = {"outputs": [{"type": "markdown", "storage": "inline", "source": html}]}

    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(metadata, metadata_file)


visualize_plot_comp = kfp.components.create_component_from_func(
    func=visualize_plot, base_image=BASE_IMAGE
)

In [54]:
@kfp.dsl.pipeline(name="Example pipeline with plot")
def pipeline():
    task = visualize_plot_comp()


run_info = kfp.Client().create_run_from_pipeline_func(pipeline, arguments={})